In [ ]:
#! pip install transformers datasets peft accelerate bitsandbytes

In [ ]:
import torch
import csv
import pandas as pd
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

In [ ]:
import pandas as pd
from datasets import Dataset


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1", load_in_4bit=True, torch_dtype=torch.float16, device_map="auto")

In [ ]:
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)
tokenizer.pad_token = "!"
CUTOFF_LEN = 256
LORA_R = 8
LORA_ALPHA = 2 * LORA_R
LORA_DROPOUT = 0.1

In [ ]:
config = LoraConfig(r=LORA_R, lora_alpha=LORA_ALPHA, target_modules=[ "w1", "w2", "w3"], lora_dropout=LORA_DROPOUT, bias="none", task_type="CAUSAL_LM")
model = get_peft_model(model, config)

In [20]:
tarot = load_dataset('barissglc/tarot', split = 'train')
df = pd.read_csv('final_perfume_data.csv', encoding='utf-8', encoding_errors='ignore')
perfume = Dataset.from_pandas(df)

In [21]:
import random

def draw_tarot_cards():
    tarot_cards = [
        "The Fool", "The Magician", "The High Priestess", "The Empress", "The Emperor", 
        "The Hierophant", "The Chariot", "The Lovers", "Strength", "The Hermit", 
        "The Wheel of Fortune", "Justice", "The Hanged Man", "Death", "Temperance", 
        "The Devil", "The Tower", "The Star", "The Moon", "The Sun", "Judgement", "The World",
        "Ace of Wands", "Two of Wands", "Three of Wands", "Four of Wands", "Five of Wands", "Six of Wands", 
        "Seven of Wands", "Eight of Wands", "Nine of Wands", "Ten of Wands", "Page of Wands", "Knight of Wands", 
        "Queen of Wands", "King of Wands",
        "Ace of Pentacles", "Two of Pentacles", "Three of Pentacles", "Four of Pentacles", "Five of Pentacles", 
        "Six of Pentacles", "Seven of Pentacles", "Eight of Pentacles", "Nine of Pentacles", "Ten of Pentacles", 
        "Page of Pentacles", "Knight of Pentacles", "Queen of Pentacles", "King of Pentacles",
        "Ace of Swords", "Two of Swords", "Three of Swords", "Four of Swords", "Five of Swords", 
        "Six of Swords", "Seven of Swords", "Eight of Swords", "Nine of Swords", "Ten of Swords", 
        "Page of Swords", "Knight of Swords", "Queen of Swords", "King of Swords",
        "Ace of Cups", "Two of Cups", "Three of Cups", "Four of Cups", "Five of Cups", 
        "Six of Cups", "Seven of Cups", "Eight of Cups", "Nine of Cups", "Ten of Cups", 
        "Page of Cups", "Knight of Cups", "Queen of Cups", "King of Cups"
    ]

    # Shuffle the list of tarot cards
    random.shuffle(tarot_cards)

    # Draw the first three cards from the shuffled list
    card1, card2, card3 = tarot_cards[:3]

    return card1, card2, card3

# Example usage:
card1, card2, card3 = draw_tarot_cards()
print(f"Card 1: {card1}")
print(f"Card 2: {card2}")
print(f"Card 3: {card3}")

Card 1: Knight of Pentacles
Card 2: Queen of Pentacles
Card 3: Three of Swords


In [ ]:
def generate_prompt(user_query):
  sys_msg = "Imagine you are a professional Perfumer, come up with the notes of perfume based on the description"
  p = "<s> [INST]" + sys_msg +"\n"+  user_query['Description'] + "[/INST] " +  str(user_query['Notes']) + "</s>"
  return p

In [31]:
tokenize = lambda prompt: tokenizer(prompt + tokenizer.eos_token, truncation=True, max_length=CUTOFF_LEN, padding="max_length")
# tarot_dataset = tarot.shuffle().map(lambda x: tokenize(generate_prompt(x)), remove_columns=['Description' , 'Notes'])
perfume_dataset = perfume.shuffle().map(lambda x: tokenize(generate_prompt(x)), remove_columns=['Description' , 'Notes'])

Map: 100%|██████████| 2191/2191 [00:01<00:00, 1345.93 examples/s]


In [33]:
trainer = Trainer(
  model=model,
  train_dataset=perfume_dataset,
  args=TrainingArguments(
    per_device_train_batch_size=40,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    learning_rate=1e-4,
    logging_steps=2,
    optim="adamw_torch",
    save_strategy="epoch",
    output_dir="/root/model"
  ),
  data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)


/workspace/tuesday/.venv/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [34]:
model.config.use_cache = False
trainer.train()

/workspace/tuesday/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
2,3.361300
4,2.947800
6,2.740400
8,2.545900
10,2.441400
12,2.357600
14,2.274200
16,2.234100
18,2.189400
20,2.200600


/workspace/tuesday/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=26, training_loss=2.4458052745232215, metrics={'train_runtime': 5083.5873, 'train_samples_per_second': 0.862, 'train_steps_per_second': 0.005, 'total_flos': 2.976603611930296e+17, 'train_loss': 2.4458052745232215, 'epoch': 1.89})

In [64]:
def generate_inference_prompt(user_query):
  sys_msg = "Imagine you are a professional Perfumer, come up with the notes of perfume based on the description"
  p = "<s> [INST]" + sys_msg +"\n"+  user_query + "[/INST]"
  return p

def generate_optimization_prompt(user_query):
  sys_msg = "Imagine you are a professional Perfumer, come up with the description of perfume based on the description tarot reading but dont use general tarot mistery association stick stricly to description and be laconic"
  p = "<s> [INST]" + sys_msg +"\n"+  user_query + "[/INST]"
  return p

In [68]:
import warnings
warnings.filterwarnings('ignore')
# description = " Could there ever be an aroma more blissful than removing a freshly-baked cake from the oven? The feeling of accomplishment for a job well done, the mouthwatering, sweet, warm smell that fills the room, the anticipation of that perfect first bite- it's heaven on earth. With Vanilla Cake, Montale invites you to enjoy that wondrous feeling every single day. This gourmand and bewitching fragrance recipe calls for blending milk, grilled almonds, warm caramel and vanilla for the ultimate meringue silllage. Creamy, sweet, toasty and rich, this is a perfect gourmand scent, the kind to leave a trail of admirers salivating. Utterly delicious."
# description = "Temperance suggests a period of balance and harmony in your life, with the Eight of Wands representing rapid movement and progress, while Strength signifies courage and inner strength. This combination of cards suggests that you are in a period of great growth and transformation, and while the journey may be difficult and unpredictable, you have the courage, strength and balance to take the steps necessary to achieve success."

Card_1 = "The Nine of Pentacles"
Card_2 = "The Chariot"
Card_3 = "The Hermit"
description = "The Nine of Pentacles indicates that you are in a place of abundance and security, and you are able to enjoy the fruits of your labor. The Chariot indicates that you are in a place of power and control, and you are able to take charge of your life and make the necessary changes to ensure that you are able to reach your goals. The Hermit indicates that you are in a place of contemplation and reflection, and you are able to take a step back and look at the bigger picture. You are able to see the opportunities that are available to you and you are able to make the necessary changes to ensure that you are able to reach your goals."

# Card_1 = "Eight of Swords"
# Card_2 = "The Lovers"
# Card_3 = "The High Priestess"
# description = "The Eight of Swords indicates that you are in a period of feeling trapped and stuck in a situation that is not allowing you to move forward. The Lovers card suggests that you are in a place of making a decision that will have a significant impact on your life. The High Priestess card indicates that you are in a place of deep contemplation and reflection, and that you are in a place of understanding the deeper meaning of the situation. You are in a place of understanding the importance of taking the time to reflect and make the right decision for yourself."

# Card_1 = "Death"
# Card_2 = "The Fool"
# Card_3 = "The King of Swords"
# description = "The Death card indicates that you are in a period of transformation and change. The Fool card indicates that you are in a place of new beginnings and that you are ready to take a leap of faith and start a new journey. The King of Swords indicates that you are in a place of clarity and understanding, and that you are able to make decisions with a clear head and a strong sense of purpose. You are in a place of strength and courage, and you are ready to take on any challenge that comes your way."
model.eval()
with torch.no_grad():
    print("Cards:\n", Card_1,",", Card_2,",", Card_3)
    print("\n\nTarot Reading: \n", description, "\n\n")
    prompt = generate_optimization_prompt(description)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(input_ids, max_length=1000)
    perfume_description = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    perfume_description = perfume_description[len(prompt)-2:]
    print("\n\nPerfume description:\n", perfume_description, "\n\n")

    # description = "The Temperance perfume is a harmonious blend of the Eight of Wands and Strength tarot cards. This fragrance captures the essence of balance and harmony, with notes of sweet honey and warm vanilla, combined with the freshness of citrus and herbs. The result is a complex and intriguing scent that is both calming and invigorating, perfect for those seeking to find their inner strength and balance. The Temperance perfume is a reminder that even in the midst of change and transformation, we have the power to stay grounded and centered, and to move forward with courage and determination."
    prompt = generate_inference_prompt(perfume_description)

    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(input_ids, max_length=1000)

    notes = tokenizer.decode(outputs[0], skip_special_tokens=True)
    notes = notes[len(prompt)-2:].strip()
    print(f"\n\nPerfume imgredients: \n {notes}", "\n\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Cards:
 The Nine of Pentacles , The Chariot , The Hermit


Tarot Reading: 
 The Nine of Pentacles indicates that you are in a place of abundance and security, and you are able to enjoy the fruits of your labor. The Chariot indicates that you are in a place of power and control, and you are able to take charge of your life and make the necessary changes to ensure that you are able to reach your goals. The Hermit indicates that you are in a place of contemplation and reflection, and you are able to take a step back and look at the bigger picture. You are able to see the opportunities that are available to you and you are able to make the necessary changes to ensure that you are able to reach your goals. 




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




Perfume description:
 The Nine of Pentacles, The Chariot, and The Hermit are combined to create a scent that is rich, powerful, and contemplative. The Nine of Pentacles brings a sense of abundance and security, while The Chariot brings a sense of power and control. The Hermit brings a sense of contemplation and reflection, allowing you to see the opportunities available to you and make the necessary changes to reach your goals. This scent is a luxurious blend of sandalwood, amber, and musk, with a hint of spice and a touch of sweetness. It is a scent that is both powerful and contemplative, perfect for those who are in a place of abundance and security and are ready to take charge of their lives and make the necessary changes to reach their goals. 




Perfume imgredients: 
 Sandalwood, amber, musk, spice, sweetness 


